In [1]:
import sys
if 'google.colab' in sys.modules:
    from google.colab import drive
    import os

    drive.mount('/content/drive')
    os.chdir('/content/drive/Shareddrives/Confidence Probes/taskdiscovery')
    !pip3 install transformer_lens fancy_einsum einops datasets encoders plotly circuitsvis

else:
    print("Not running in Google Colab, skipping installation of requirements.")

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.5/594.5 kB 57.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 69.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.7/739.7 kB 60.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.7/40.7 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 86.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.2 MB/s eta 0:

In [2]:
from transformer_lens import HookedTransformer
from data import load_ICL_dataset,word_pairs_to_prompt_data,create_prompt,FVDataset
from torch.utils.data import DataLoader
from tqdm import tqdm
from torch.nn import functional as F
from functools import partial
import numpy as np
import torch
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.cluster import KMeans


In [3]:
model_name = "gpt2-small"
device = torch.device("cuda")
model = HookedTransformer.from_pretrained(model_name, device=device)
tokenizer = model.tokenizer

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Loaded pretrained model gpt2-small into HookedTransformer


In [5]:
d = {"prompt":[], "label": [], "task" : []}
for task in os.listdir("function_vectors_data/abstractive"):
    task = task.replace(".json", "")
    dataset = load_ICL_dataset(task)
    word_pairs = dataset['train'][:5]
    for test_pair in dataset['test']:
        prompt_data = word_pairs_to_prompt_data(word_pairs, query_target_pair=test_pair, prepend_bos_token=True)
        sentence = create_prompt(prompt_data)
        d["prompt"].append(sentence)
        d["label"].append(test_pair["output"])
        d["task"].append(task)

df = pd.DataFrame(d)
df_subset = df[df["task"].isin(["antonym", "capitalize"])]
df_subset

,prompt,label,task
1596,<|endoftext|>Q: dance\nA: Dance\n\nQ: soda\nA:...,Mantis,capitalize
1597,<|endoftext|>Q: dance\nA: Dance\n\nQ: soda\nA:...,Cow,capitalize
1598,<|endoftext|>Q: dance\nA: Dance\n\nQ: soda\nA:...,Without,capitalize
1599,<|endoftext|>Q: dance\nA: Dance\n\nQ: soda\nA:...,Ostrich,capitalize
1600,<|endoftext|>Q: dance\nA: Dance\n\nQ: soda\nA:...,That,capitalize
...,...,...,...
2265,<|endoftext|>Q: hardware\nA: software\n\nQ: fa...,unilateral,antonym
2266,<|endoftext|>Q: hardware\nA: software\n\nQ: fa...,peace,antonym
2267,<|endoftext|>Q: hardware\nA: software\n\nQ: fa...,width,antonym
2268,<|endoftext|>Q: hardware\nA: software\n\nQ: fa...,reduction,antonym


In [6]:
df.task.unique()

array(['ag_news', 'capitalize', 'antonym', 'capitalize_first_letter',
       'capitalize_last_letter', 'capitalize_second_letter',
       'commonsense_qa', 'country-capital', 'country-currency',
       'english-french', 'english-german', 'english-spanish',
       'landmark-country', 'lowercase_first_letter',
       'lowercase_last_letter', 'national_parks', 'next_item',
       'next_capital_letter', 'park-country', 'person-instrument',
       'person-occupation', 'person-sport', 'prev_item', 'present-past',
       'product-company', 'sentiment', 'singular-plural', 'synonym',
       'word_length'], dtype=object)

In [7]:
tokenized_data = df_subset['prompt'].apply(lambda x: tokenizer(x, max_length=64, truncation=True, padding='max_length', return_tensors="pt")).to_numpy()
dataset = FVDataset(tokenized_data, df_subset['task'].to_numpy())

batch_size = 2
dl = DataLoader(dataset, batch_size=batch_size, shuffle=False,drop_last=True)

In [145]:
tokenizer.batch_decode(next(iter(dl))[0])

/content/drive/Shareddrives/Confidence Probes/taskdiscovery/data.py:557: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(input_ids, dtype=torch.long)
/content/drive/Shareddrives/Confidence Probes/taskdiscovery/data.py:558: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask = torch.tensor(attention_mask, dtype=torch.long)


['<|endoftext|>Q: dance\nA: Dance\n\nQ: soda\nA: Soda\n\nQ: before\nA: Before\n\nQ: youthful\nA: Youthful\n\nQ: orange\nA: Orange\n\nQ: mantis\nA:<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|>']

In [8]:
model.cfg.use_attn_result = True
n_layers = 12


def save_(save_to, result, hook):
  save_to.append(result)
  return result

def attn_filter(name):
  return name.endswith("attn.hook_result")

def pre_filter(name):
  return name.endswith("resid_pre")


attention_results = 0
all_activations = []
tasks = []

for batch in tqdm(dl):
  with torch.no_grad():
    attn_storage = []
    act_storage = []
    attn_mask =  batch[1]
    # batch x tokens x vocab size
    output = model.run_with_hooks(
                    batch[0],
                    attention_mask = batch[1],
                    fwd_hooks=[(attn_filter,partial(save_, attn_storage),) ] +
                    [(pre_filter,partial(save_, act_storage),) ]

            )

    # will return a tensor with shape batch size with indices of last token
    last_token_idx = attn_mask.cumsum(1).argmax(1)

    activations = torch.stack(act_storage)[:, torch.arange(batch_size), last_token_idx, :]
    all_activations.append(activations.cpu())
    attention_results += (torch.stack(attn_storage).cpu())
    tasks += list(batch[2])

    #print(list(batch[2]), tokenizer.batch_decode(output[torch.arange(batch_size), last_token_idx,:].argmax(-1)))


# n_layers x 1 x tokens x heads x d_model
print()
print(attention_results.shape )
mean_attn = (attention_results/len(dl)).mean(1).to(device)
all_activations = torch.cat(all_activations, 1)
print(all_activations.shape)

  0%|          | 0/337 [00:00<?, ?it/s]/content/drive/Shareddrives/Confidence Probes/taskdiscovery/data.py:557: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(input_ids, dtype=torch.long)
/content/drive/Shareddrives/Confidence Probes/taskdiscovery/data.py:558: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask = torch.tensor(attention_mask, dtype=torch.long)
100%|██████████| 337/337 [00:23<00:00, 14.19it/s]


torch.Size([12, 2, 64, 12, 768])
torch.Size([12, 674, 768])


In [9]:
for layer in range(n_layers):

  x = all_activations[layer]
  kmeans = KMeans(n_clusters=2,n_init =10, random_state=0).fit(x)

  label_encoder = LabelEncoder()
  true_labels = label_encoder.fit_transform(tasks)

  predicted_labels = kmeans.labels_

  if np.mean(predicted_labels == true_labels) < 0.5:
      predicted_labels = 1 - predicted_labels

  # Calculate "accuracy"
  accuracy = accuracy_score(true_labels, predicted_labels)

  print(f"Layer {layer} accuracy: {accuracy}")

Layer 0 accuracy: 1.0
Layer 1 accuracy: 0.9970326409495549
Layer 2 accuracy: 0.9970326409495549
Layer 3 accuracy: 0.9925816023738873
Layer 4 accuracy: 1.0
Layer 5 accuracy: 1.0
Layer 6 accuracy: 1.0
Layer 7 accuracy: 1.0
Layer 8 accuracy: 1.0
Layer 9 accuracy: 1.0
Layer 10 accuracy: 1.0
Layer 11 accuracy: 1.0


In [178]:
mean_attn.shape

torch.Size([12, 64, 12, 768])

In [31]:
attention_scores = []


# n_heads x tokens x n_heads x dim
# [i,:,i,:] will be 1s

mask = torch.eye(12,device = device).unsqueeze(1).unsqueeze(-1).bool()
mask = mask.expand(-1, 64, -1, 768).repeat_interleave(2,dim=0)

def attn_mean_ablation(mean, result, hook):
  result[-n_layers * batch_size:] = torch.where(mask, mean.repeat(n_layers* batch_size,1,1,1).to(device), result[-n_layers* batch_size:])
  return result


def increase_batch(act,hook):
  return torch.cat([act,act[:batch_size].repeat(n_layers,1,1)],0)

def attn_filter_layer(i,name):
  return name.endswith(f"{i}.attn.hook_result")

def pre_filter(name):
  return name.endswith("resid_pre")


idx = 0

kls = []
tasks = []

for batch in tqdm(dl):
  with torch.no_grad():

    output = model.run_with_hooks(
                    batch[0],
                    attention_mask = batch[1],
                    fwd_hooks=[(partial(attn_filter_layer,i),partial(attn_mean_ablation, mean_attn[i]),)  for i in range(n_layers)] +
                    [(pre_filter,increase_batch)]
            )

    attn_mask = batch[1]
    last_token_idx = attn_mask.cumsum(1).argmax(1)
    output = output.unflatten(0,(-1,batch_size))
    output = output[:,torch.arange(batch_size), last_token_idx,:]

    kl_loss = F.kl_div(output[1:].softmax(-1).log(), output[0:1].softmax(-1), reduction = 'none').sum(-1).T

    kls.append(kl_loss.cpu().numpy())
    tasks.append(df_subset["task"].to_numpy()[idx])
    # print(tokenizer.decode(batch[0].squeeze()) ,tokenizer.decode(output.argmax(-1)[0]))

    idx += 1





100%|██████████| 337/337 [02:01<00:00,  2.77it/s]


In [41]:
tasks = [item for sublist in tasks for item in sublist]

In [43]:
len(tasks)

2869

In [42]:
kmeans = KMeans(n_clusters=2, random_state=0).fit(np.concatenate(kls,0))

label_encoder = LabelEncoder()
true_labels = label_encoder.fit_transform(tasks)
predicted_labels = kmeans.labels_


if np.mean(predicted_labels == true_labels) < 0.5:
    predicted_labels = 1 - predicted_labels

# Calculate "accuracy"
accuracy = accuracy_score(true_labels, predicted_labels)

print(f"Accuracy: {accuracy}")

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


ValueError: operands could not be broadcast together with shapes (674,) (2869,) 

In [44]:
predicted_labels

array([0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1,
       0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [159]:

# first 12 will be for batch 1 second 12 will be for batch 2 ...

torch.cat([act,act[:batch_size].repeat(n_layers,1,1)],0)

# When multiplying the activations we are multiplying by n_heads so it will be batch times n_heads
# this means that for the mask we want something that has shape [batch * n_heads x tokens x  ]
# so for the first batchsize in mask we want all of them to have first head equal to 1 and rest 0


NameError: name 'act' is not defined

In [166]:
(mask[0] == mask[1]).sum()

tensor(589824, device='cuda:0')

In [168]:
(mask[0] == mask[3]).sum()

tensor(491520, device='cuda:0')

In [158]:
mask

tensor([[[[ True,  True,  True,  ...,  True,  True,  True],
          [False, False, False,  ..., False, False, False],
          [False, False, False,  ..., False, False, False],
          ...,
          [False, False, False,  ..., False, False, False],
          [False, False, False,  ..., False, False, False],
          [False, False, False,  ..., False, False, False]],

         [[ True,  True,  True,  ...,  True,  True,  True],
          [False, False, False,  ..., False, False, False],
          [False, False, False,  ..., False, False, False],
          ...,
          [False, False, False,  ..., False, False, False],
          [False, False, False,  ..., False, False, False],
          [False, False, False,  ..., False, False, False]],

         [[ True,  True,  True,  ...,  True,  True,  True],
          [False, False, False,  ..., False, False, False],
          [False, False, False,  ..., False, False, False],
          ...,
          [False, False, False,  ..., False, False,